Consider a problem adapted from [ProjectEuler.net](https://projecteuler.net/problem=27):

$$
n^2 + n + 41
$$

This formula is discoverd by the great mathematician Euler. It is amazing that it generates 40 primes for $0 \leq n \leq 39$. There is another fomula $n^2 - 79n + 1601$ generates 80 primes for $0 \leq n \leq 79$.

Consider quadratics of the form:
$$
n^2 + an + b, |a|\lt 1000, |b|\leq 1000
$$

Find the values of the coefficients,  $a$ and $b$, for the quadratic expression that produces the maximum number of primes for consecutive values of , $n$ starting with $n=0$.


Naively, we can seperate the problems into smaller pieces problem, and solve each piece, then re-ensemble them.

The smaller pieces might be:
1. prime test (we've done)
2. how many prime is generated from a particular quadratic
3. find which quadratic generates the most primes from all given quadratics

In [8]:
def is_prime(n):
    if n < 2: # changed to include negative integers
        return False
    d = 2
    while d*d <= n:
        if n%d == 0:
            return False
        d += 1
    return True

The problem "how many prime is generated from a particular quadratic" can be further divide into:

1. calculate value of quadratic given values of others
2. how to do the repetition to count prime

In [2]:
def quadratic(n, a, b):
    return n**2 + n*a + b

We've coded the `quadratic` but we're not sure whether **it works as expected**. We may make use of `is_prime` to check given integer is prime or not

In [3]:
quadratic(10,10,10) == 10**2 + 10*10 + 10

True

In [4]:
assert(is_prime(quadratic(0,1,41)))
assert(is_prime(quadratic(2,1,41)))
assert(is_prime(quadratic(39,1,41)))

Okay, it seems good. Then we proceed to "how to count how many primes is generated from given quadratic in consecutive integers". Indeed, it is question of "how to enumerate/ iterate all possible cases".

In [5]:
a = 1
b = 41
n = 0
while is_prime(quadratic(n, a, b)):
    n = n+1
n

40

For now, we've figured out how to find the count. We might go for generalising the counting process into a **function** (ie.: abstracting). 

```
a = 1 # this can be parameter
b = 41 # this can be parameter
n = 0
while is_prime(quadratic(n, a, b)):
    n = n+1
n # as return value
```

In [6]:
def quadratic_prime_count(a,b):
    n = 0
    while is_prime(quadratic(n, a, b)):
        n += 1
    return n
print(quadratic_prime_count(1,41))
print(quadratic_prime_count(-79,1601))

40
80


The question states that

$$
n^2 + an + b, |a|\lt 1000, |b|\leq 1000
$$

Then, it is also another question of enumerating and iterating all $a$, $b$ to find the most primal quadratics.

In [7]:
a = -999
max_a = 1
max_b = 41
max_n = 40
while a < 1000:
    b = -1000
    while b <= 1000:
        if quadratic_prime_count(a,b) > max_n:
            max_a = a 
            max_b = b
            max_n = quadratic_prime_count(max_a,max_b)
        b += 1
    a += 1
max_a,max_b,max_n

(-61, 971, 71)

Indeed, -61 and 971 is the answer. Now, we may refactor the code abit.

In [34]:
def is_prime(n):
    if n < 2: # changed to include negative integers
        return False
    d = 2
    while d*d <= n:
        if n%d == 0:
            return False
        d += 1
    return True

def quadratic(n, a, b):
    return n**2 + n*a + b

def quadratic_prime_count(a,b):
    n = 0
    while is_prime(quadratic(n, a, b)):
        n += 1
    return n

max_a = 1
max_b = 41
max_n = 40
for a in range(-999, 1000):
    for b in range(-1000, 1000):
        if quadratic_prime_count(a,b) > max_n:
            max_a = a 
            max_b = b
            max_n = quadratic_prime_count(max_a,max_b)
max_a,max_b,max_n

(-61, 971, 71)

# Benefit of functions

Below is equivalent code that do the same thing without using many functions.

In [19]:
max_a = 1
max_b = 41
max_n = 40
a = -999
while a < 1000:
    b = -1000
    while b <= 1000:
        n = 0
        is_prime_bool = True
        # count primes generated by quadratic
        while is_prime_bool:
            quad = n**2 + n*a + b
            # check if p is prime
            if quad < 2:
                is_prime_bool = False
            divisor = 2
            while divisor*divisor <= quad:
                if quad%divisor == 0:
                    is_prime_bool = False
                    break
                divisor += 1
            n += 1
        # max value check
        if n > max_n:
            max_a = a 
            max_b = b
            max_n = n
        b += 1
    a += 1
max_a,max_b,max_n

(-61, 971, 72)

Some radicals (if there is any) might argue the code above as a whole that it is much clearer that how the program is going. But writing the code this way have some disadvantages.

1. Error Prone
2. Inflexible

1. Error Prone. The code below will stuck forever, could you find the bug?
```
max_a = 1
max_b = 41
max_n = 40
a = -999
while a < 1000:
    b = -1000
    while b <= 1000:
        n = 0
        is_prime_bool = True
        # count primes generated by quadratic
        if quad < 2:
            is_prime_bool = False
        while is_prime_bool:
            quad = n**2 + n*a + b
            # check if quad is prime
            divisor = 2
            while divisor*divisor <= quad:
                if quad%divisor == 0:
                    is_prime_bool = False
                    break
                divisor += 1
        n += 1
        # max value check
        if n > max_n:
            max_a = a 
            max_b = b
            max_n = n
        b += 1
    a += 1
max_a,max_b,max_n
```

Indeed, it turns out that `n += 1` is mis-indented.

The code below is also stuck forever. Then we may debug each function. It turns out that `is_prime` doesn't work as expected for negative integer `n`.

```
 def is_prime(n):
        d = 2
        while d*d <= n:
            if n%d == 0:
                return False
            d += 1
        return True

def quadratic(n, a, b):
    return n**2 + n*a + b

def quadratic_prime_count(a,b):
    n = 0
    while is_prime(quadratic(n, a, b)):
        n += 1
    return n
a = -999
max_a = 1
max_b = 41
max_n = 40
while a < 1000:
    b = -1000
    while b <= 1000:
        if quadratic_prime_count(a,b) > max_n:
            max_a = a 
            max_b = b
            max_n = quadratic_prime_count(max_a,max_b)
        b += 1
    a += 1
max_a,max_b,max_n
```

2. Inflexibility

Suppose that we have a more effective way to do primality test.

In [32]:
def sieve(n):
    li = [True for _ in range(n)]
    li[0] = False
    li[1] = False
    for i in range(2, n):
        if li[i] == True:
            for j in range(i*i, n, i):
                li[j] = False
    return li

memo = sieve(100000)
def is_prime(n):
    if n < 2:
        return False
    return memo[n]

If we use function, then we not need to change other part of code after we adopt the new primality test algorithm. The code is flexible. In contrary, we have to change the solution code in the less-function code.

In [33]:
def quadratic(n, a, b):
    return n**2 + n*a + b

def quadratic_prime_count(a,b):
    n = 0
    while is_prime(quadratic(n, a, b)):
        n += 1
    return n
a = -999
max_a = 1
max_b = 41
max_n = 40
while a < 1000:
    b = -1000
    while b <= 1000:
        if quadratic_prime_count(a,b) > max_n:
            max_a = a 
            max_b = b
            max_n = quadratic_prime_count(max_a,max_b)
        b += 1
    a += 1
max_a,max_b,max_n

(-61, 971, 71)

## side note:

Generate and test all possible cases is known as **Brute Force** algorithm. It is not definitely the fastest but it always give the **optimal** answer. Furthermore, it is the easiest algorithm can be thought to solve any problem; it is a naive algorithm.

# Conclusion

The use of functions can make our program more organized and easier to be changed.

Besides, the task seperations and re-ensemble are one of the way to solve complex problem. To solve a complex problem, we can solve smaller pieces of the complex problem. I think this method is naive and natural that we apply it natural in real world. 

Indeed, as we see in the code, the function name also convey what it does, thus more readable. 

Every function try to do single thing, this is similar to the principle of single responsibility. Although looking at each of the functions itself seems trivial, they together can form the complex program.

Here is some guidelines in designing functions:
1. Function suppress implementation details
2. Function generalizes things
3. **Consider required parameters, return values**
4. Exploit the fact that arguments are local to the functions
5. Exploit the fact that variables inside a function is local to the current function
6. Code function over common patterns

## Side Note: Python Scoping

Python has its custom scoping rule to follow, you should read the online documentation or read LEGB rule.

# Exercise

You make an account on [ProjectEuler.net](https://projecteuler.net/). Then solve the [problem 6](https://projecteuler.net/problem=6)